In [3]:
# importing required libraries
from pathlib import Path

import torch
from torch import nn
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.nn.utils.prune as prune

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import Normalize
from torchmetrics import Accuracy

import torch.optim as optim
from cleverhans.torch.attacks.projected_gradient_descent import (projected_gradient_descent)

import quantus
import captum
from captum.attr import Saliency, IntegratedGradients, NoiseTunnel

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
import copy
import gc

import warnings
warnings.filterwarnings('ignore')

In [37]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [39]:
transformer = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

# downloading dataset from pytorch
train_dataset = datasets.FashionMNIST(root='./datasets', train=True, download = True, transform=transformer)
test_dataset = datasets.FashionMNIST(root='./datasets', train=False, download = True, transform=transformer)

# preparing the dataloader
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True) # num_workers=4,
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, pin_memory=True)

In [41]:
%run models.ipynb

In [45]:
model = LeNet()
learning_rate = 0.001
epochs = 50 
# loss function
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
# adam Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [47]:
def train_model(model, epochs):
    model.train()
    for epoch in range(epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            # passing data through the model
            output = model(x_batch)
            # calculating loss
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()

        # Evaluating model in each epoch
        if epochs%10==0:
            prediction, labels = evaluate_model(model, test_dataloader, device)
            test_acc = np.mean(np.argmax(prediction.cpu().numpy(), axis=1) == labels.cpu().numpy())
            print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [49]:
model_normal = train_model(model = model.to(device), epochs = epochs)

Epoch 1/50 - test accuracy: 82.53% and CE loss 0.38
Epoch 2/50 - test accuracy: 85.22% and CE loss 0.56
Epoch 3/50 - test accuracy: 87.23% and CE loss 0.36
Epoch 4/50 - test accuracy: 87.84% and CE loss 0.16
Epoch 5/50 - test accuracy: 87.98% and CE loss 0.24
Epoch 6/50 - test accuracy: 88.65% and CE loss 0.24
Epoch 7/50 - test accuracy: 88.83% and CE loss 0.22
Epoch 8/50 - test accuracy: 88.47% and CE loss 0.15
Epoch 9/50 - test accuracy: 89.29% and CE loss 0.16
Epoch 10/50 - test accuracy: 89.52% and CE loss 0.38
Epoch 11/50 - test accuracy: 89.65% and CE loss 0.15
Epoch 12/50 - test accuracy: 89.80% and CE loss 0.23
Epoch 13/50 - test accuracy: 89.33% and CE loss 0.10
Epoch 14/50 - test accuracy: 89.40% and CE loss 0.11
Epoch 15/50 - test accuracy: 89.62% and CE loss 0.12
Epoch 16/50 - test accuracy: 89.73% and CE loss 0.36
Epoch 17/50 - test accuracy: 88.88% and CE loss 0.21
Epoch 18/50 - test accuracy: 89.79% and CE loss 0.13
Epoch 19/50 - test accuracy: 89.69% and CE loss 0.20
Ep

In [51]:
# Model to GPU 
model_normal.to(device)
# Evaluation Mode
model_normal.eval()
# Check test set performance.
prediction, labels = evaluate_model(model_normal, test_dataloader, device)
test_acc = np.mean(np.argmax(prediction.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 89.05%


In [10]:
model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "lenet_fmnist.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_normal.state_dict(), f=model_save_path)

Saving the model: models\lenet_fmnist.pth


# Train Adversarial Models

In [9]:
model = LeNet()
learning_rate = 1e-4
epochs = 40
eps = [0.01,0.03,0.06,0.0,0.3,0.5]
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
def train_adv(model, epsilon, epochs):
    model.train()
    eps = epsilon
    for epoch in range(epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            x_batch = projected_gradient_descent(model, x_batch, eps, eps/10, 40, np.inf)
            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()

        # Evaluating Model after each epoch
        if epochs%10==0:
            prediction, labels = evaluate_model(model, test_dataloader, device)
            test_acc = np.mean(np.argmax(prediction.cpu().numpy(), axis=1) == labels.cpu().numpy())
            print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [15]:
model_adversarial = train_adv(model=model.to(device),
                    epsilon = eps[0], 
                    epochs=epochs)

Epoch 1/40 - test accuracy: 72.75% and CE loss 0.65
Epoch 2/40 - test accuracy: 74.71% and CE loss 0.60
Epoch 3/40 - test accuracy: 77.28% and CE loss 0.52
Epoch 4/40 - test accuracy: 78.62% and CE loss 0.60
Epoch 5/40 - test accuracy: 79.47% and CE loss 0.57
Epoch 6/40 - test accuracy: 81.42% and CE loss 0.22
Epoch 7/40 - test accuracy: 82.02% and CE loss 0.53
Epoch 8/40 - test accuracy: 83.04% and CE loss 0.55
Epoch 9/40 - test accuracy: 83.57% and CE loss 0.40
Epoch 10/40 - test accuracy: 84.38% and CE loss 0.25
Epoch 11/40 - test accuracy: 84.24% and CE loss 0.47
Epoch 12/40 - test accuracy: 85.49% and CE loss 0.26
Epoch 13/40 - test accuracy: 85.57% and CE loss 0.65
Epoch 14/40 - test accuracy: 85.63% and CE loss 0.33
Epoch 15/40 - test accuracy: 85.96% and CE loss 0.25
Epoch 16/40 - test accuracy: 85.97% and CE loss 0.35
Epoch 17/40 - test accuracy: 86.23% and CE loss 0.35
Epoch 18/40 - test accuracy: 86.60% and CE loss 0.25
Epoch 19/40 - test accuracy: 86.61% and CE loss 0.55
Ep

In [21]:
model_adversarial.to(device)
model_adversarial.eval()

# Check test set performance.
prediction, labels = evaluate_model(model_adversarial, test_dataloader, device)
test_acc = np.mean(np.argmax(prediction.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 87.97%


In [23]:
model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "lenet_fmnist_adv.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_adversarial.state_dict(), f=model_save_path)

Saving the model: models\lenet_fmnist_adv.pth


# L1 Unstructured 

In [53]:
model = LeNet()
learning_rate = 0.001
epochs = 50
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr = learning_rate)

In [69]:
%run utils.ipynb

In [60]:
orig_params = count_params(model)
print(f"Unpruned LeNet-5 model has {orig_params} trainable parameters")

Unpruned LeNet-5 model has 44426 trainable parameters


In [62]:
for layer, param in model.named_parameters():
    print(f"layer.name: {layer} & param.shape = {param.shape}")

layer.name: conv_1.weight & param.shape = torch.Size([6, 1, 5, 5])
layer.name: conv_1.bias & param.shape = torch.Size([6])
layer.name: conv_2.weight & param.shape = torch.Size([16, 6, 5, 5])
layer.name: conv_2.bias & param.shape = torch.Size([16])
layer.name: fc_1.weight & param.shape = torch.Size([120, 256])
layer.name: fc_1.bias & param.shape = torch.Size([120])
layer.name: fc_2.weight & param.shape = torch.Size([84, 120])
layer.name: fc_2.bias & param.shape = torch.Size([84])
layer.name: fc_3.weight & param.shape = torch.Size([10, 84])
layer.name: fc_3.bias & param.shape = torch.Size([10])


In [64]:
for layer_name in model.state_dict().keys():
    print(layer_name, model.state_dict()[layer_name].shape)

conv_1.weight torch.Size([6, 1, 5, 5])
conv_1.bias torch.Size([6])
conv_2.weight torch.Size([16, 6, 5, 5])
conv_2.bias torch.Size([16])
fc_1.weight torch.Size([120, 256])
fc_1.bias torch.Size([120])
fc_2.weight torch.Size([84, 120])
fc_2.bias torch.Size([84])
fc_3.weight torch.Size([10, 84])
fc_3.bias torch.Size([10])


In [66]:
model.state_dict().keys()

odict_keys(['conv_1.weight', 'conv_1.bias', 'conv_2.weight', 'conv_2.bias', 'fc_1.weight', 'fc_1.bias', 'fc_2.weight', 'fc_2.bias', 'fc_3.weight', 'fc_3.bias'])

In [71]:
print(f"LeNet-5 global sparsity = {compute_sparsity_lenet(model):.2f}%")

LeNet-5 global sparsity = 0.00%


In [73]:
# Using l1 unstructured pruning to prune the modles
for name, module in model.named_modules():
    # prune 20% of weights in for all hidden layaers
    if isinstance(module, torch.nn.Conv2d):
        prune.l1_unstructured(module = module, name = 'weight', amount = 0.2)
    
    # prune 10% of weights for output layer
    elif isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module = module, name = 'weight', amount = 0.1)

In [75]:
print(f"LeNet-5 global sparsity = {compute_sparsity_lenet(model):.2f}%")


LeNet-5 global sparsity = 10.58%


In [77]:
new_params = count_params(model)
print(f"Pruned LeNet-5 model has {new_params} trainable parameters")

Pruned LeNet-5 model has 44426 trainable parameters


In [67]:
model_l1_unstructured = train_model(model = model.to(device), epochs = epochs)

Epoch 1/50 - test accuracy: 80.86% and CE loss 0.29
Epoch 2/50 - test accuracy: 84.95% and CE loss 0.73
Epoch 3/50 - test accuracy: 86.20% and CE loss 0.45
Epoch 4/50 - test accuracy: 86.38% and CE loss 0.49
Epoch 5/50 - test accuracy: 87.75% and CE loss 0.29
Epoch 6/50 - test accuracy: 88.38% and CE loss 0.38
Epoch 7/50 - test accuracy: 87.91% and CE loss 0.29
Epoch 8/50 - test accuracy: 88.56% and CE loss 0.16
Epoch 9/50 - test accuracy: 89.02% and CE loss 0.23
Epoch 10/50 - test accuracy: 88.87% and CE loss 0.21
Epoch 11/50 - test accuracy: 89.00% and CE loss 0.33
Epoch 12/50 - test accuracy: 89.57% and CE loss 0.15
Epoch 13/50 - test accuracy: 89.64% and CE loss 0.10
Epoch 14/50 - test accuracy: 89.56% and CE loss 0.18
Epoch 15/50 - test accuracy: 90.08% and CE loss 0.40
Epoch 16/50 - test accuracy: 89.93% and CE loss 0.44
Epoch 17/50 - test accuracy: 89.92% and CE loss 0.27
Epoch 18/50 - test accuracy: 89.54% and CE loss 0.19
Epoch 19/50 - test accuracy: 89.43% and CE loss 0.35
Ep

In [68]:
# Model to GPU and eval mode.
model_l1_unstructured.to(device)
model_l1_unstructured.eval()

# Check test set performance.
prediction, labels = evaluate_model(model_l1_unstructured, test_dataloader, device)
test_acc = np.mean(np.argmax(prediction.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 88.95%


In [69]:
model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "lenet_fmnist_l1_unstructured.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_l1_unstructured.state_dict(), f=model_save_path)

Saving the model: models\lenet_fmnist_l1_unstructured.pth


# Global Pruning

In [56]:
model = LeNet()
learning_rate = 0.001
epochs = 20
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(params=model.parameters(), lr = learning_rate)

In [57]:
# parameters to prune for global pruning
parameters_to_prune = (
    (model.conv_1, 'weight'),
    (model.conv_2, 'weight'),
    (model.fc_1, 'weight'),
    (model.fc_2, 'weight'),
    (model.fc_3, 'weight')
)

In [58]:
print(f"LeNet-5 global sparsity = {compute_sparsity_lenet(model):.2f}%")

LeNet-5 global sparsity = 0.00%


In [59]:
prune_rates_global = [0.2, 0.3, 0.4, 0.5, 0.6]

In [61]:
def train_pruned(model, epochs):
    # Prunes the model once
    for iter_prune_round in range(1):
        print(f"\n\nIterative Global pruning round = {iter_prune_round + 1}")
        
        # Global Pruning
        prune.global_unstructured(
            parameters_to_prune,
            pruning_method = prune.L1Unstructured,
            amount = prune_rates_global[iter_prune_round]
            
        )
    
        # Print current global sparsity level
        print(f"LeNet-5 global sparsity = {compute_sparsity_lenet(model):.2f}%")
        
        
        # Fine-training loop
        print("\nFine-tuning pruned model to recover model's performance\n")
        model.train()
        for epoch in range(epochs):
            for x_batch, y_batch in train_dataloader:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                output = model(x_batch)
                loss = criterion(output, y_batch)
                loss.backward()
                optimizer.step()
    
            # Evaluate model!
            if epochs%10==0:
                prediction, labels = evaluate_model(model, test_dataloader, device)
                test_acc = np.mean(np.argmax(prediction.cpu().numpy(), axis=1) == labels.cpu().numpy())
                print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [62]:
model_global = train_pruned(model = model.to(device), epochs = epochs)



Iterative Global pruning round = 1
LeNet-5 global sparsity = 20.00%

Fine-tuning pruned model to recover model's performance

Epoch 1/20 - test accuracy: 83.03% and CE loss 0.28
Epoch 2/20 - test accuracy: 85.60% and CE loss 0.24
Epoch 3/20 - test accuracy: 87.07% and CE loss 0.14
Epoch 4/20 - test accuracy: 88.25% and CE loss 0.45
Epoch 5/20 - test accuracy: 88.33% and CE loss 0.17
Epoch 6/20 - test accuracy: 88.39% and CE loss 0.22
Epoch 7/20 - test accuracy: 89.36% and CE loss 0.12
Epoch 8/20 - test accuracy: 88.65% and CE loss 0.59
Epoch 9/20 - test accuracy: 89.53% and CE loss 0.19
Epoch 10/20 - test accuracy: 89.51% and CE loss 0.02
Epoch 11/20 - test accuracy: 89.58% and CE loss 0.29
Epoch 12/20 - test accuracy: 88.65% and CE loss 0.27
Epoch 13/20 - test accuracy: 89.43% and CE loss 0.15
Epoch 14/20 - test accuracy: 89.09% and CE loss 0.12
Epoch 15/20 - test accuracy: 89.62% and CE loss 0.19
Epoch 16/20 - test accuracy: 89.70% and CE loss 0.10
Epoch 17/20 - test accuracy: 89.5

In [63]:
model_global.to(device)
model_global.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_global, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 89.47%


In [64]:
model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "lenet_fmnist_global.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_global.state_dict(), f=model_save_path)

Saving the model: models\lenet_fmnist_global.pth


# Layered Structured

In [66]:
model = LeNet()
learning_rate = 0.01
epochs = 20
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr = learning_rate)

In [67]:
def train_pruned(model, epochs):
    for iter_prune_round in range(1):
        print(f"\n\nIterative Global pruning round = {iter_prune_round + 1}")
        
        # Prune layer-wise using layered structured pruning
        prune.ln_structured(model.conv_1, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.conv_2, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.fc_1, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.fc_2, name = "weight", amount = 0.1, n = 2, dim = 0)
        prune.ln_structured(model.fc_3, name = "weight", amount = 0, n = 2, dim = 0)
        
        # Print current global sparsity level
        print(f"LeNet-5 global sparsity = {compute_sparsity_lenet(model):.2f}%")
        
        
        # Fine-training loop
        print("\nFine-tuning pruned model to recover model's performance\n")
        model.train()
        for epoch in range(epochs):
            for x_batch, y_batch in train_dataloader:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                output = model(x_batch)
                loss = criterion(output, y_batch)
                loss.backward()
                optimizer.step()
    
            # Evaluate model!
            if epochs%10==0:
                prediction, labels = evaluate_model(model, test_dataloader, device)
                test_acc = np.mean(np.argmax(prediction.cpu().numpy(), axis=1) == labels.cpu().numpy())
                print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [68]:
model_layered_structured = train_pruned(model = model.to(device), epochs = epochs)



Iterative Global pruning round = 1
LeNet-5 global sparsity = 9.86%

Fine-tuning pruned model to recover model's performance

Epoch 1/20 - test accuracy: 82.60% and CE loss 0.34
Epoch 2/20 - test accuracy: 83.46% and CE loss 0.60
Epoch 3/20 - test accuracy: 83.72% and CE loss 0.33
Epoch 4/20 - test accuracy: 84.48% and CE loss 0.70
Epoch 5/20 - test accuracy: 84.59% and CE loss 0.29
Epoch 6/20 - test accuracy: 84.32% and CE loss 0.56
Epoch 7/20 - test accuracy: 84.64% and CE loss 0.22
Epoch 8/20 - test accuracy: 85.00% and CE loss 0.41
Epoch 9/20 - test accuracy: 86.04% and CE loss 0.37
Epoch 10/20 - test accuracy: 85.69% and CE loss 0.44
Epoch 11/20 - test accuracy: 84.98% and CE loss 0.75
Epoch 12/20 - test accuracy: 85.03% and CE loss 0.21
Epoch 13/20 - test accuracy: 85.89% and CE loss 0.40
Epoch 14/20 - test accuracy: 84.35% and CE loss 0.31
Epoch 15/20 - test accuracy: 85.54% and CE loss 0.42
Epoch 16/20 - test accuracy: 85.49% and CE loss 0.31
Epoch 17/20 - test accuracy: 85.54

In [69]:
model_layered_structured.to(device)
model_layered_structured.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_layered_structured, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 85.99%


In [70]:
model_path = Path("models")
model_path.mkdir(parents=True, exist_ok=True)

model_name = "lenet_fmnist_structured.pth"
model_save_path = model_path / model_name

print(f"Saving the model: {model_save_path}")
torch.save(obj=model_layered_structured.state_dict(), f=model_save_path)

Saving the model: models\lenet_fmnist_structured.pth
